# TASCA M6 T01 Regressió

In [1]:
# import libraries
import numpy as np
from numpy import arange
from numpy import std
from numpy import absolute
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [2]:
# import dataset
columnes= ['CRIM', 'ZN', 'INDUS','CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
# la variable dependent
columna_y = ['MEDV']
housing = pd.read_csv('C:\housing_data.csv', names = columnes, header= None, encoding = "ISO-8859-1" )

- CRIM -> taxa de criminalitat per cápita per ciutat.
- ZN -> proporció de sòl residencial zonificat per a lots de més de 25.000 metres cuadrats
- INDUS -> proporció d'hectàrees comercials no minoristes per ciutat
- CHAS -> variable ficticia de Charles River (=1 si el tram limita el riu; 0 en cas contrari)
- NOX -> concentració d'òxids nítrics (parts per 10 milions)
- RM -> nombre mitjà d'habitacions per habitatge
- AGE -> proporció d'unitats ocupades pels propietaris construïdes abans de 1940.
- DIS -> distàncices ponderades fins a cinc centres de treball de boston
- RAD -> índex d'accessibilitat a les carreteres radials.
- TAX -> Taxa de l'impost sobre béns inmobles de valor total per 10.000 dólars
- PTRATIO -> ràtio alumnes professor per municipi.
- B -> 1000(BK - 0.63)^2 on BK és la proporció de negres per ciutat.
- LSTAT -> % d'estatus inferior de la població.
- MEDV -> Valor mitjà de les cases opcupades pel propietari per 1000 dolars.

## Preprocessament

In [3]:
housing.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## elements duplicats

In [4]:
# miro si hi ha elements duplicats
housing_clean = housing.drop_duplicates()
if len(housing_clean) == len(housing):
    print("No hi ha elements duplicats")
else:
    print("Hi ha elements duplicats")

No hi ha elements duplicats


## elements nuls

In [5]:
# comprovo si hi ha elements nuls
housing.isnull()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,False,False,False,False,False,False,False,False,False,False,False,False,False,False
502,False,False,False,False,False,False,False,False,False,False,False,False,False,False
503,False,False,False,False,False,False,False,False,False,False,False,False,False,False
504,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
# comprovo si hi ha elements nuls.
(housing.isnull().sum())*100 / len(housing)

CRIM       0.0
ZN         0.0
INDUS      0.0
CHAS       0.0
NOX        0.0
RM         0.0
AGE        0.0
DIS        0.0
RAD        0.0
TAX        0.0
PTRATIO    0.0
B          0.0
LSTAT      0.0
MEDV       0.0
dtype: float64

## dummies

In [7]:
housing_dummy = housing
# faig la variable dummy
housing_dummy_CHAS= pd.get_dummies(housing["CHAS"], prefix = "CHAS")

# agreguem les columnes
housing_dummy = pd.concat([housing_dummy, housing_dummy_CHAS], axis = 1)
housing_dummy = housing_dummy.rename(columns={'CHAS_1' : 'RIVER_LIMIT', 'CHAS_0' : 'NO_RIVER'})
housing_dummy_1 = housing_dummy.drop(["CHAS"], axis = 1)
housing_dummy_1

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,NO_RIVER,RIVER_LIMIT
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0,1,0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6,1,0
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7,1,0
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4,1,0
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4,1,0
502,0.04527,0.0,11.93,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6,1,0
503,0.06076,0.0,11.93,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9,1,0
504,0.10959,0.0,11.93,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0,1,0


In [8]:
columnes_amb_dummy = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT','MEDV','NO_RIVER','RIVER_LIMIT']
# les variables independents
columnes_X = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT','NO_RIVER','RIVER_LIMIT']
# per l'exercici 5 trio les variables independents sense 'RM'
columnes_X_exercici_5 = ['CRIM', 'ZN', 'INDUS','CHAS', 'NOX', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT','NO_RIVER','RIVER_LIMIT']

## estandarització

In [9]:
# defineixo scaler
scaler = RobustScaler()
# transformo
df = scaler.fit_transform(housing_dummy_1)
# converteixo l'array en DataFrame
housing_scaled_df = pd.DataFrame(df, columns=columnes_amb_dummy)
# mostro resultat
housing_scaled_df.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,NO_RIVER,RIVER_LIMIT
0,-0.069593,1.44,-0.571650,0.000000,0.496612,-0.250765,0.285777,-0.20,-0.087855,-1.339286,0.261902,-0.637681,0.351097,0.0,0.0
1,-0.063755,0.00,-0.202943,-0.394286,0.287940,0.028542,0.569789,-0.15,-0.227390,-0.446429,0.261902,-0.221889,0.050157,0.0,0.0
2,-0.063760,0.00,-0.202943,-0.394286,1.323171,-0.334353,0.569789,-0.15,-0.227390,-0.446429,0.066675,-0.732634,1.692790,0.0,0.0
3,-0.062347,0.00,-0.581720,-0.457143,1.069783,-0.646279,0.924391,-0.10,-0.279070,-0.125000,0.153016,-0.841579,1.529781,0.0,0.0
4,-0.052144,0.00,-0.581720,-0.457143,1.271680,-0.475025,0.924391,-0.10,-0.279070,-0.125000,0.261902,-0.602699,1.880878,0.0,0.0


### Creem variables dependents i independents

In [10]:
# labels són els valors que volem predir
y = housing_scaled_df['MEDV'].values
# eliminem labels de les features
X = housing_scaled_df.drop('MEDV', axis=1)
# guardem features
feature_list = list(X.columns)
# convertim en numpy array
X = X.values

## Exercici 1
Crea almenys dos models de regressió diferents per intentar predir el millor possible preu de les vivendes (MEDV) DE l'arxiu adjunt.

In [11]:
# Regressió Random Forest
# triem el tipos de model
rf = RandomForestRegressor(n_estimators = 300, random_state = 30)
# ajustem
rf.fit(X,y)
# fem les prediccions
rf_pred = rf.predict(X)
# triem r2
r2_rf = r2_score(y, rf_pred)
print(r2_rf)
# treiem MSE
mse_rf = mean_squared_error(y, rf_pred)
print(mse_rf)

0.9827554314483801
0.022889379203935814


In [12]:
# Regressió Elastic Net
# triem el tipus de model
model_enet = ElasticNet(alpha = 0.10)
# ajustem
model_enet.fit(X, y)
# fem les prediccions
pred_enet = model_enet.predict(X)
# treiem el r2
r2_enet = r2_score(y, pred_enet)
print(r2_enet)
# treiem MSE
mse_enet = mean_squared_error(y, pred_enet)
print(mse_enet)

0.6795937892889988
0.4252874888871227


## Exercici 2
Comparal's en base al MSE i al R2.

In [13]:
print(
'Si comparem R2 dels regressors: random forest té:', r2_rf,'model elastic net té:', r2_enet)
print(
'Si comparem MSE random forest té:', mse_rf,'model elastic net té:', mse_enet)

Si comparem R2 dels regressors: random forest té: 0.9827554314483801 model elastic net té: 0.6795937892889988
Si comparem MSE random forest té: 0.022889379203935814 model elastic net té: 0.4252874888871227


## Exercici 3
Entrenal's utilitzant els diferents paràmetres que admeten per intentar millorar-ne la predicció.

### Creem conjunts d'entrenament i prova

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 10)
print(X_train.shape); print(X_test.shape)

(404, 14)
(102, 14)


### Entrenament Random Forest

In [15]:
rf = RandomForestRegressor(n_estimators = 300, random_state = 30)
# ajustem entrenament
model_rf = rf.fit(X_train, y_train)
# predim entrenament
pred_train_rf = rf.predict(X_train)
# imprimim mètriques d'entrenament
r2_rf_train = r2_score(y_train, pred_train_rf).round(2)
mse_rf_train = mean_squared_error(y_train, pred_train_rf)
print('Dades entrenament')
print(r2_rf_train)
print(mse_rf_train)
# predim avaluació
pred_test_rf = rf.predict(X_test)
# imprimim métriques d'avaluació
r2_rf_test = r2_score(y_test, pred_test_rf)
mse_rf_test = mean_squared_error(y_test, pred_test_rf).round(2)
print('Dades evaluació')
print(r2_rf_test)
print(mse_rf_test)

Dades entrenament
0.98
0.024130431955745494
Dades evaluació
0.8560902178371823
0.24


In [16]:
# mirem els paràmetres que estem utilitzant:
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_jobs': None, 'oob_score': False, 'random_state': 30, 'verbose': 0, 'warm_start': False}


In [17]:
# intentem millorar el model i fem RandomizedSearch per buscar els millors hiperparàmetres
# número d'arbres
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num=10)]
# número de features a considerar a cada divisió
max_features = ['auto', 'sqrt']
# número màxim de nivells a l'arbre
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# número mínim de mostres requerides per dividir node
min_samples_split = [2,5,10]
# número mínim de mostres requerides per cada fulla de node
min_samples_leaf = [1,2,4]
# mètode de selecció de mostres per entrenar cada arbre
bootstrap = [True, False]
# creem la xarxa aleatòria
random_grid = {'n_estimators' : n_estimators,
              'max_features' : max_features,
              'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf,
              'bootstrap' : bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [18]:
# busquem els millors hiperparàmetres
# utilitzem 5 camps de validació creuada
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                              n_iter = 100, cv = 5, verbose = 2, random_state = 42,
                              n_jobs = -1)
rf_random.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_estimators=300,
                                                   random_state=30),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [19]:
# aquests són els millors paràmetres per millorar el model.
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [20]:
rf = RandomForestRegressor(n_estimators = 400,
                           min_samples_split = 2,
                           min_samples_leaf = 1,
                           max_features = 'sqrt',
                           max_depth = None,
                           bootstrap = False)
# ajustem entrenament
model_rf = rf.fit(X_train, y_train)
# predim entrenament
pred_train_rf = rf.predict(X_train)
# imprimim mètriques d'entrenament
r2_rf_train = r2_score(y_train, pred_train_rf)
mse_rf_train = mean_squared_error(y_train, pred_train_rf)
print('Dades entrenament')
print(r2_rf_train)
print(mse_rf_train)
# predim avaluació
pred_test_rf = rf.predict(X_test)
# imprimim métriques d'avaluació
r2_rf_test = r2_score(y_test, pred_test_rf)
mse_rf_test = mean_squared_error(y_test, pred_test_rf)
print('Dades evaluació')
print(r2_rf_test)
print(mse_rf_test)

Dades entrenament
1.0
5.247423029125126e-29
Dades evaluació
0.8459069879560884
0.2533805554287043


 Un cop aplicat el tuneig d'hiperparàmetres veiem que les dades de l'score d'entrenament milloren de 0,98 a 1, per tant arribem al nivell òptim, i les dades d'avaluació pasen d' R2 0,86 a R2 0,84 i MSE de 0,24 a MSE 0,26

### Entrenament Regressió Elastic Net

In [21]:
model = ElasticNet()
# definim el model del métode d'avaluació per buscar els millors hiperparàmetres
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state = 1)
# definim la xarxa
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = arange(0, 1, 0.01)
# definim la recerca
search = GridSearchCV(model, grid, scoring = 'neg_mean_absolute_error', cv=cv, n_jobs = -1)
# fem la recerca
results = search.fit(X_train, y_train)
# imprimim
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -0.399
Config: {'alpha': 0.1, 'l1_ratio': 0.0}


C:\Users\imber\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.023e+01, tolerance: 4.889e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [22]:
# triem el tipus de model
model_enet = ElasticNet(alpha = 0.01, l1_ratio = 0.0)
# ajustem entrenament
model = model_enet.fit(X_train,y_train)
# predim entrenament
pred_train_enet = model_enet.predict(X_train)
# imprimim métriques d'entrenament
r2_enet_train = r2_score(y_train, pred_train_enet)
mse_enet_train = mean_squared_error(y_train, pred_train_enet)
print('Dades entrenament')
print(r2_enet_train)
print(mse_enet_train)
# predim avaluació
pred_test_enet = model_enet.predict(X_test)
# imprimim métriques d'avaluació
r2_enet_test = r2_score(y_test, pred_test_enet)
mse_enet_test = mean_squared_error(y_test, pred_test_enet)
print('Dades evaluació')
print(r2_enet_test)
print(mse_enet_test)

Dades entrenament
0.7478139426726093
0.30516135821287066
Dades evaluació
0.6700980787049817
0.5424693238581625


C:\Users\imber\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.450e+01, tolerance: 4.889e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


## Exercici 4
Compara el seu rendiment emprant l'aproximació train/test o emprant totes les dades(validació interna).

##### Random Forest

In [23]:
print('R2 emprant validació interna té un valor de:', r2_rf,'amb entrenament el conjunt entrenament té un valor de:,',r2_rf_train,'i evaluació:',r2_rf_test)

R2 emprant validació interna té un valor de: 0.9827554314483801 amb entrenament el conjunt entrenament té un valor de:, 1.0 i evaluació: 0.8459069879560884


In [24]:
print('MSE emprant validació interna té un valor de:',mse_rf,'amb entrenament el conjunt entrenament té un valor de:,',mse_rf_train,'i evaluació:',mse_rf_test)

MSE emprant validació interna té un valor de: 0.022889379203935814 amb entrenament el conjunt entrenament té un valor de:, 5.247423029125126e-29 i evaluació: 0.2533805554287043


##### Elastic Net

In [25]:
print('R2 emprant validació interna té un valor de:', r2_enet,'amb entrenament el conjunt entrenament té un valor de:,',r2_enet_train,'i evaluació:',r2_enet_test)

R2 emprant validació interna té un valor de: 0.6795937892889988 amb entrenament el conjunt entrenament té un valor de:, 0.7478139426726093 i evaluació: 0.6700980787049817


In [26]:
print('MSE emprant validació interna té un valor de:', mse_enet,'amb entrenament el conjunt entrenament té un valor de:,',mse_enet_train,'i evaluació:', mse_enet_test)

MSE emprant validació interna té un valor de: 0.4252874888871227 amb entrenament el conjunt entrenament té un valor de:, 0.30516135821287066 i evaluació: 0.5424693238581625


## Exercici 5
No facis servir la variable del nombre d'habitacions (RM) a l'hora de fer prediccions.

In [27]:
# labels són els valors que volem predir
y = housing_scaled_df['MEDV'].values
# eliminem labels de les features i la variable que tampoc volem
X_2 = housing_scaled_df.drop(['MEDV','RM'], axis=1)
# guardem features
feature_list = list(X_2.columns)
# convertim en numpy array
X_2 = X_2.values

In [28]:
# Creem els nous conjunts d'entrenament i evaluació sense la variable independent RM
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y, train_size = 0.7, test_size = 0.3, random_state = 10)
print(X_train_2.shape); print(X_test_2.shape)

(354, 13)
(152, 13)


#### Random Forest sense RM 

In [29]:
# Regressió Random Forest
# triem el tipos de model
rf = RandomForestRegressor(n_estimators = 300, random_state = 30)
# ajustem
rf.fit(X_2,y)
# fem les prediccions
rf_pred_2 = rf.predict(X_2)
# treiem r2
r2_rf_2 = r2_score(y, rf_pred_2)
print('R2:',r2_rf_2)
# treiem MSE
mse_rf_2 = mean_squared_error(y, rf_pred_2)
print('mse:',mse_rf_2)

R2: 0.9767682227974511
mse: 0.03083643156271022


In [30]:
print('respecte a lexercici 1 veiem que: pasem dun r2 de',r2_rf,'a',r2_rf_2,'i un mse de',mse_rf,'a',mse_rf_2,'per tant el model empitjora')

respecte a lexercici 1 veiem que: pasem dun r2 de 0.9827554314483801 a 0.9767682227974511 i un mse de 0.022889379203935814 a 0.03083643156271022 per tant el model empitjora


In [31]:
rf = RandomForestRegressor(n_estimators = 400,
                           min_samples_split = 2,
                           min_samples_leaf = 1,
                           max_features = 'sqrt',
                           max_depth = None,
                           bootstrap = False)
# ajustem entrenament
model_rf = rf.fit(X_train_2, y_train_2)
# predim entrenament
pred_train_rf_2 = rf.predict(X_train_2)
# imprimim mètriques d'entrenament
r2_rf_train_2 = r2_score(y_train_2, pred_train_rf_2)
mse_rf_train_2 = mean_squared_error(y_train_2, pred_train_rf_2)
print('Dades entrenament')
print(r2_rf_train_2)
print(mse_rf_train_2)
# predim avaluació
pred_test_rf_2 = rf.predict(X_test_2)
# imprimim métriques d'avaluació
r2_rf_test_2 = r2_score(y_test_2, pred_test_rf_2)
mse_rf_test_2 = mean_squared_error(y_test_2, pred_test_rf_2)
print('Dades evaluació')
print(r2_rf_test_2)
print(mse_rf_test_2)

Dades entrenament
1.0
5.422364548773908e-29
Dades evaluació
0.8040855551806427
0.3007487212490667


In [32]:
print('entrenant el model veiem que pasem dun R2 de',r2_rf_test,'a',r2_rf_test_2,'i un mse de:',mse_rf_test,'a',mse_rf_test_2)

entrenant el model veiem que pasem dun R2 de 0.8459069879560884 a 0.8040855551806427 i un mse de: 0.2533805554287043 a 0.3007487212490667


- mirem el segon model de regressió elastic net sense RM

In [33]:
# Regressió Elastic Net
# triem el tipus de model
model_enet = ElasticNet(alpha = 0.10)
# ajustem
model_enet.fit(X_2, y)
# fem les prediccions
pred_enet_2 = model_enet.predict(X_2)
# treiem el r2
r2_enet_2 = r2_score(y, pred_enet_2).round(2)
print(r2_enet_2)
# treiem MSE
mse_enet_2 = mean_squared_error(y, pred_enet_2).round(2)
print(mse_enet_2)

0.61
0.52


In [34]:
# triem el tipus de model
model_enet = ElasticNet(alpha = 0.10)
# ajustem entrenament
model_2 = model_enet.fit(X_train_2,y_train_2)
# predim entrenament
pred_train_enet_2 = model_enet.predict(X_train_2)
# imprimim métriques d'entrenament
r2_enet_train_2 = r2_score(y_train_2, pred_train_enet_2).round(2)
mse_enet_train_2 = mean_squared_error(y_train_2, pred_train_enet_2).round(2)
print('Dades entrenament')
print(r2_enet_train_2)
print(mse_enet_train_2)
# predim avaluació
pred_test_enet_2 = model_enet.predict(X_test_2)
# imprimim métriques d'avaluació
r2_enet_test_2 = r2_score(y_test_2, pred_test_enet_2).round(2)
mse_enet_test_2 = mean_squared_error(y_test_2, pred_test_enet_2).round(2)
print('Dades evaluació')
print(r2_enet_test_2)
print(mse_enet_test_2)

Dades entrenament
0.63
0.46
Dades evaluació
0.54
0.71


In [35]:
print('entrenant el model veiem que pasem dun R2 de',r2_enet_test,'a',r2_enet_test_2,'i un mse de:',mse_enet_test,'a',mse_enet_test_2)

entrenant el model veiem que pasem dun R2 de 0.6700980787049817 a 0.54 i un mse de: 0.5424693238581625 a 0.71
